In [64]:
import pandas as pd
import numpy as np
from tqdm import tqdm_notebook as tqdm
from torch.optim.optimizer import Optimizer
import matplotlib.pyplot as plt
from copy import deepcopy
import numpy as np
import random
import torch
from transformers import pipeline
import warnings 
warnings.filterwarnings('ignore')
from pytorch_lightning import seed_everything
from torch.utils.data import DataLoader
import os
import gc
gc.collect()

def get_jaccard_sim(str1, str2): 
    a = set(str1.split()) 
    b = set(str2.split())
    c = a.intersection(b)
    return float(len(c)) / (len(a) + len(b) - len(c))

In [65]:
def set_seed(seed = int):
    '''Sets the seed of the entire notebook so results are the same every time we run.
    This is for REPRODUCIBILITY.'''
    np.random.seed(seed)
    random_state = np.random.RandomState(seed)
    random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    os.environ['PYTHONHASHSEED'] = str(seed)
    seed_everything(seed)
    return random_state
random_state = set_seed(42)

Global seed set to 42


In [66]:
from transformers import DistilBertTokenizerFast
from transformers import DistilBertForQuestionAnswering

tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')
model = DistilBertForQuestionAnswering.from_pretrained("distilbert-base-uncased")
model.load_state_dict(torch.load('Product_Data_SQuAD_model_2144.pt'))
model.eval()
nlp = pipeline('question-answering', model=model.to('cpu'), tokenizer=tokenizer)
gc.collect()

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForQuestionAnswering: ['vocab_projector.weight', 'vocab_projector.bias', 'vocab_layer_norm.weight', 'vocab_transform.bias', 'vocab_layer_norm.bias', 'vocab_transform.weight']
- This IS expected if you are initializing DistilBertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForQuestionAnswering were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['qa_outputs.weight', 'qa_outputs.bias']
You should probably TRAIN this mode

244

In [67]:
def model_predict(nlp,df):
    table = pd.DataFrame()
    idx_list = sorted(df.index.tolist())
    for i in tqdm(idx_list):
        sample = df.loc[[i]]
        string_X_train = sample['string_X_train'].values[0]
        QA_input = {
            'question': 'What is the product name?',
            'context': string_X_train
        }
        res = nlp(QA_input)
        predict = QA_input['context'][res['start']:res['end']]
        row = pd.DataFrame({'predict:':predict},index=[i])
        table = table.append(row)
    return table

In [68]:
import pandas as pd
from tqdm import tqdm_notebook as tqdm


df = pd.read_excel('台塑企業_ 產品寶典20210303.xlsx',engine='openpyxl')
display(df.head(3))
display(df[df['品名']=='MA'])

產品集合 = set(df['品名'].values)

'''
train_df = pd.read_csv('Train_Product_Data_2021_0114.csv',index_col=0).drop(['Forward','Backward','label_for_train'],axis=1)
train_df = train_df.dropna(axis=0)
display(train_df.head(3))

val_df = pd.read_csv('Val_Product_Data_2021_0114.csv',index_col=0).drop(['Forward','Backward'],axis=1)
val_df = val_df.dropna(axis=0)
display(val_df.head(3))
'''

,RIGID,分機,公司代號,公司事業部門,品名,新增
0,黃淑玲,7190,11,台塑塑膠部,CAUSTIC SODA LIQUID FLAKE PEARLS MICROPEARLS,NaN
1,黃淑玲,7190,11,台塑塑膠部,CAUSTIC SODA LIQUID,NaN
2,黃淑玲,7190,11,台塑塑膠部,COMMODITY FORMOSACN,NaN


,RIGID,分機,公司代號,公司事業部門,品名,新增
531,NaN,NaN,2P,南亞塑四部化學品部,MA,NaN


"\ntrain_df = pd.read_csv('Train_Product_Data_2021_0114.csv',index_col=0).drop(['Forward','Backward','label_for_train'],axis=1)\ntrain_df = train_df.dropna(axis=0)\ndisplay(train_df.head(3))\n\nval_df = pd.read_csv('Val_Product_Data_2021_0114.csv',index_col=0).drop(['Forward','Backward'],axis=1)\nval_df = val_df.dropna(axis=0)\ndisplay(val_df.head(3))\n"

In [69]:
品名2部門 = dict(zip(df['品名'],df['公司事業部門']))
品名2代號 = dict(zip(df['品名'],df['公司代號']))

In [70]:
val_df = pd.read_csv('preprocess_for_SQUAD_wordninja.csv',index_col=0)[['45A','Y_label','EXPNO']]
val_df.columns = ['string_X_train','Y_label','EXPNO']
val_df

,string_X_train,Y_label,EXPNO
0,MASS PVC RESIN B-57 QUANTITY 175 MT AT 1300 US...,MASS PVC RESIN B-57,11
1,PHTHALIC ANHYDRIDE PA QUANTITY 306 MT UNIT PRI...,PHTHALIC ANHYDRIDE,27
2,COMMODITY LLDPE TAISOX 3470 QUANTITY 320 MT 2 ...,LLDPE TAISOX,18
3,COMMODITY AMOUNT POLYESTER TEXTURED YARN HS CO...,POLYESTER PARTIALY ORIENTED YARN,25
4,COMMODITY AMOUNT POLYESTER TEXTURED YARN HS CO...,POLYESTER FILAMENT YARN,25
...,...,...,...
3609,COMMODITY MONO ETHYLENE GLYCOL x 000 D QUANTIT...,MONO ETHYLENE GLYCOL,2A
3610,CFR KOBE JAPAN x 000 D VISCOSE RAYON STAPLE FI...,VISCOSE RAYON STAPLE FIBER,41
3611,DESCRIPTION OF GOODS QUANTITY MT UNIT PRICE US...,PC RESIN,4A
3612,DESCRIPTION OF GOODS QUANTITY MT UNIT PRICE US...,PC RESIN,4A


In [71]:
train_df = val_df

# 如果品名是單詞的話 前後加個空白

In [72]:
for i in df.index:
    if ' ' not in df.loc[i,'品名']:
        name = df.loc[i,'品名']
        df.loc[i,'品名'] = f' {name} '
        assert df.loc[i,'品名'][0] == ' '
        assert df.loc[i,'品名'][-1] == ' '

In [73]:
display(df[df['品名']=='MA'])
display(df[df['品名']==' MA '])

,RIGID,分機,公司代號,公司事業部門,品名,新增


,RIGID,分機,公司代號,公司事業部門,品名,新增
531,NaN,NaN,2P,南亞塑四部化學品部,MA,NaN


In [74]:
產品集合 = set(df['品名'].values)

# find_fail_sample and drop fail_sample

In [75]:
def find_fail_sample(df):
    fails = []
    for i in df.index:
        context = df.loc[i,'string_X_train']
        answer = df.loc[i,'Y_label']
        if str(answer) not in str(context):
            fails.append(i)
    return fails
train_fails = find_fail_sample(train_df)
val_fails = find_fail_sample(val_df)
print(train_fails,val_fails)
display(val_df.loc[val_fails])
print(val_df.shape)
val_df = val_df.drop(val_fails,axis=0)
print(val_df.shape)

[182, 521, 745, 748, 758, 759, 760, 829, 832, 833, 875, 876, 891, 908, 920, 923, 925, 957, 987, 998, 1003, 1004, 1007, 1009, 1046, 1048, 1083, 1132, 1135, 1136, 1154, 1203, 1209, 1266, 1301, 1315, 1317, 1326, 1368, 1398, 1481, 1501, 1502, 1512, 1552, 1553, 1589, 1602, 1643, 1650, 1677, 1679, 1680, 1713, 1714, 1716, 1718, 1721, 1754, 1755, 1780, 1831, 1871, 1872, 1962, 1964, 1965, 2067, 2208, 2210, 2218, 2229, 2233, 2246, 2282, 2353, 2387, 2389, 2393, 2478, 2485, 2502, 2541, 2553, 2573, 2574, 2596, 2600, 2608, 2680, 2709, 2720, 2768, 2826, 2828, 2833, 2838, 2839, 2840, 2841, 2844, 2847, 2912, 2962, 2985, 2986, 3000, 3024, 3026, 3083, 3084, 3133, 3162, 3166, 3194, 3215, 3216, 3225, 3235, 3248, 3295, 3296, 3339, 3393, 3394, 3399, 3554, 3584, 3587, 3590, 3591, 3595, 3605, 3608] [182, 521, 745, 748, 758, 759, 760, 829, 832, 833, 875, 876, 891, 908, 920, 923, 925, 957, 987, 998, 1003, 1004, 1007, 1009, 1046, 1048, 1083, 1132, 1135, 1136, 1154, 1203, 1209, 1266, 1301, 1315, 1317, 1326, 1368, 

,string_X_train,Y_label,EXPNO
182,PHTHALIC ANHYDRIDE 504 MT AT USD 1010 M Tx 000...,NaN,27
521,PHTHALIC ANHYDRIDE 504 MT AT USD 980 M Tx 000 ...,NaN,27
745,9000 MT PHTHALIC ANHYDRIDE PAx 000 DAT USD 825...,NaN,27
748,9000 MT PHTHALIC ANHYDRIDE PAx 000 DAT USD 825...,NaN,27
758,9000 MT PHTHALIC ANHYDRIDE PAx 000 DAT USD 825...,NaN,27
...,...,...,...
3590,3600 MT 2 F CL x 000 D PHTHALIC ANHYDRIDE x 00...,NaN,27
3591,PHTHALIC ANHYDRIDE PAx 000 D Q TY 54 MTS x 000...,NaN,27
3595,108 MT 6 F CL OF PHTHALIC ANHYDRIDE AT 98000 P...,NaN,27
3605,108 MT 6 F CL OF PHTHALIC ANHYDRIDE AT USD 960...,NaN,27


(3253, 3)
(3119, 3)


In [76]:
def Collection_method(df,產品集合):
    labels = {}
    for i in tqdm(df.index):
        products = []
        for p in 產品集合:
            if p in df.loc[i,'string_X_train']:
                products.append(p)
        labels[i] = products
    predict = pd.DataFrame(index=labels.keys(),columns=['predict'])
    predict['predict'] = labels.values()
    return predict
predict = Collection_method(val_df,產品集合)
result = val_df.join(predict)

  0%|          | 0/3119 [00:00<?, ?it/s]

In [77]:
result

,string_X_train,Y_label,EXPNO,predict
0,MASS PVC RESIN B-57 QUANTITY 175 MT AT 1300 US...,MASS PVC RESIN B-57,11,"[PVC RESIN, RESIN , PVC RESIN B-57]"
1,PHTHALIC ANHYDRIDE PA QUANTITY 306 MT UNIT PRI...,PHTHALIC ANHYDRIDE,27,"[PHTHALIC ANHYDRIDE , PA , PHTHALIC ANHYDRIDE..."
2,COMMODITY LLDPE TAISOX 3470 QUANTITY 320 MT 2 ...,LLDPE TAISOX,18,"[LLDPE TAISOX, TAISOX ]"
3,COMMODITY AMOUNT POLYESTER TEXTURED YARN HS CO...,POLYESTER PARTIALY ORIENTED YARN,25,"[POLYESTER TEXTURED YARN, POLYESTER PARTIALY O..."
4,COMMODITY AMOUNT POLYESTER TEXTURED YARN HS CO...,POLYESTER FILAMENT YARN,25,"[POLYESTER TEXTURED YARN, POLYESTER PARTIALY O..."
...,...,...,...,...
3609,COMMODITY MONO ETHYLENE GLYCOL x 000 D QUANTIT...,MONO ETHYLENE GLYCOL,2A,"[MONO ETHYLENE GLYCOL, ETHYLENE , INA ]"
3610,CFR KOBE JAPAN x 000 D VISCOSE RAYON STAPLE FI...,VISCOSE RAYON STAPLE FIBER,41,"[ RAYON , VISCOSE RAYON STAPLE FIBER, RAYON ST..."
3611,DESCRIPTION OF GOODS QUANTITY MT UNIT PRICE US...,PC RESIN,4A,"[PC RESIN, RESIN , INA ]"
3612,DESCRIPTION OF GOODS QUANTITY MT UNIT PRICE US...,PC RESIN,4A,"[PC RESIN, RESIN , INA ]"


In [84]:
not_find = []
for j,i in enumerate(result.iloc[:,3].values):
    if len(i) == 0:
        not_find.append(j)
len(not_find)

275

In [85]:
not_find_df = result.iloc[not_find]
not_find_df

,string_X_train,Y_label,EXPNO,predict
51,PVC SUSPENSION S65 - 112 MT AT USD 133000 PER ...,EA,11,[]
203,TERMS OF PRICE CFR AQABA PORT - J O R D A N x ...,DOP,27,[]
204,TERMS OF PRICE CFR AQABA PORT - J O R D A N x ...,DOP,27,[]
241,TOTAL 36000 KGS - 10 PCT POM FORMOCON x 000 D ...,PE,1P,[]
244,TOTM 32 MT AT USD 2420 PER M Tx 000 D CFR JAPA...,TOTM,27,[]
...,...,...,...,...
3556,ABS AG 15 AA - H x 000 D Q U A N T I T Y 18000...,ABS,4A,[]
3566,ABS AG 15 AA - H x 000 D Q U A N T I T Y 25200...,ABS,4A,[]
3596,EPICHLOROHYDRIN x 000 D Q TY 18400 x 000 DAS P...,EPICHLOROHYDRIN,1A,[]
3603,ACETONITRILE 128 M Tx 000 DAT USD 3650 PER M T...,ACETONITRILE,1A,[]


In [86]:
bert_predict = model_predict(nlp,not_find_df)
bert_predict

  0%|          | 0/275 [00:00<?, ?it/s]

,predict:
51,PVC SUSPENSION
203,DOP
204,DOP
241,BELGIUM
244,TOTM
...,...
3556,ABS
3566,ABS
3596,EPICHLOROHYDRIN
3603,ACETONITRILE


In [87]:
def get_jaccard_sim(str1, str2): 
    a = set(str1.split()) 
    b = set(str2.split())
    c = a.intersection(b)
    return float(len(c)) / (len(a) + len(b) - len(c))

In [89]:
result.iloc[not_find,3] = [ [str(i)] for i in bert_predict['predict:'].values]
result.iloc[not_find]

,string_X_train,Y_label,EXPNO,predict
51,PVC SUSPENSION S65 - 112 MT AT USD 133000 PER ...,EA,11,[PVC SUSPENSION]
203,TERMS OF PRICE CFR AQABA PORT - J O R D A N x ...,DOP,27,[DOP]
204,TERMS OF PRICE CFR AQABA PORT - J O R D A N x ...,DOP,27,[DOP]
241,TOTAL 36000 KGS - 10 PCT POM FORMOCON x 000 D ...,PE,1P,[BELGIUM]
244,TOTM 32 MT AT USD 2420 PER M Tx 000 D CFR JAPA...,TOTM,27,[TOTM]
...,...,...,...,...
3556,ABS AG 15 AA - H x 000 D Q U A N T I T Y 18000...,ABS,4A,[ABS]
3566,ABS AG 15 AA - H x 000 D Q U A N T I T Y 25200...,ABS,4A,[ABS]
3596,EPICHLOROHYDRIN x 000 D Q TY 18400 x 000 DAS P...,EPICHLOROHYDRIN,1A,[EPICHLOROHYDRIN]
3603,ACETONITRILE 128 M Tx 000 DAT USD 3650 PER M T...,ACETONITRILE,1A,[ACETONITRILE]


In [90]:
get_jaccard_sim('MASS PVC RESIN B-57','PVC RESIN B-57')

0.75

In [91]:
for j in result.loc[51,'predict']:
    print(j)

PVC SUSPENSION


In [92]:
def get_acc(df):
    correct = []
    correct_label = []
    for i in df.index:
        jacs = []
        for j in df.loc[i,'predict']:
            jacs.append(get_jaccard_sim(df.loc[i,'Y_label'],j))
        if max(jacs) >= 0.75:
            correct.append('yes')
        else:
            correct.append('no')
    return correct

In [93]:
def get_jac(df):
    all_jacs = []
    for i in df.index:
        jacs = []
        for j in df.loc[i,'predict']:
            jacs.append(get_jaccard_sim(df.loc[i,'Y_label'],j))
        all_jacs.append(max(jacs))
    return np.sum(all_jacs)/len(all_jacs)

In [94]:
correct = get_acc(result)
result['correct'] = correct
result

,string_X_train,Y_label,EXPNO,predict,correct
0,MASS PVC RESIN B-57 QUANTITY 175 MT AT 1300 US...,MASS PVC RESIN B-57,11,"[PVC RESIN, RESIN , PVC RESIN B-57]",yes
1,PHTHALIC ANHYDRIDE PA QUANTITY 306 MT UNIT PRI...,PHTHALIC ANHYDRIDE,27,"[PHTHALIC ANHYDRIDE , PA , PHTHALIC ANHYDRIDE...",yes
2,COMMODITY LLDPE TAISOX 3470 QUANTITY 320 MT 2 ...,LLDPE TAISOX,18,"[LLDPE TAISOX, TAISOX ]",yes
3,COMMODITY AMOUNT POLYESTER TEXTURED YARN HS CO...,POLYESTER PARTIALY ORIENTED YARN,25,"[POLYESTER TEXTURED YARN, POLYESTER PARTIALY O...",yes
4,COMMODITY AMOUNT POLYESTER TEXTURED YARN HS CO...,POLYESTER FILAMENT YARN,25,"[POLYESTER TEXTURED YARN, POLYESTER PARTIALY O...",yes
...,...,...,...,...,...
3609,COMMODITY MONO ETHYLENE GLYCOL x 000 D QUANTIT...,MONO ETHYLENE GLYCOL,2A,"[MONO ETHYLENE GLYCOL, ETHYLENE , INA ]",yes
3610,CFR KOBE JAPAN x 000 D VISCOSE RAYON STAPLE FI...,VISCOSE RAYON STAPLE FIBER,41,"[ RAYON , VISCOSE RAYON STAPLE FIBER, RAYON ST...",yes
3611,DESCRIPTION OF GOODS QUANTITY MT UNIT PRICE US...,PC RESIN,4A,"[PC RESIN, RESIN , INA ]",yes
3612,DESCRIPTION OF GOODS QUANTITY MT UNIT PRICE US...,PC RESIN,4A,"[PC RESIN, RESIN , INA ]",yes


In [95]:
result['correct'].value_counts()

yes    2704
no      415
Name: correct, dtype: int64

In [96]:
result['correct'].value_counts()['yes']/len(val_df)

0.8669445335043283

In [97]:
get_jac(result)

0.8881143223560665

In [98]:
部門_lst = []
for p_lst in tqdm(result['predict'].values):
    p = max(p_lst,key=len)
    jac_dict = {}
    for i in 品名2代號.keys():
        jac_dict[i] = get_jaccard_sim(i,p)
    部門_lst.append(品名2代號[max(jac_dict, key=jac_dict.get)])

  0%|          | 0/3119 [00:00<?, ?it/s]

In [99]:
result['預測部門代號'] = 部門_lst

In [100]:
result

,string_X_train,Y_label,EXPNO,predict,correct,預測部門代號
0,MASS PVC RESIN B-57 QUANTITY 175 MT AT 1300 US...,MASS PVC RESIN B-57,11,"[PVC RESIN, RESIN , PVC RESIN B-57]",yes,11
1,PHTHALIC ANHYDRIDE PA QUANTITY 306 MT UNIT PRI...,PHTHALIC ANHYDRIDE,27,"[PHTHALIC ANHYDRIDE , PA , PHTHALIC ANHYDRIDE...",yes,27
2,COMMODITY LLDPE TAISOX 3470 QUANTITY 320 MT 2 ...,LLDPE TAISOX,18,"[LLDPE TAISOX, TAISOX ]",yes,18
3,COMMODITY AMOUNT POLYESTER TEXTURED YARN HS CO...,POLYESTER PARTIALY ORIENTED YARN,25,"[POLYESTER TEXTURED YARN, POLYESTER PARTIALY O...",yes,25
4,COMMODITY AMOUNT POLYESTER TEXTURED YARN HS CO...,POLYESTER FILAMENT YARN,25,"[POLYESTER TEXTURED YARN, POLYESTER PARTIALY O...",yes,25
...,...,...,...,...,...,...
3609,COMMODITY MONO ETHYLENE GLYCOL x 000 D QUANTIT...,MONO ETHYLENE GLYCOL,2A,"[MONO ETHYLENE GLYCOL, ETHYLENE , INA ]",yes,2A
3610,CFR KOBE JAPAN x 000 D VISCOSE RAYON STAPLE FI...,VISCOSE RAYON STAPLE FIBER,41,"[ RAYON , VISCOSE RAYON STAPLE FIBER, RAYON ST...",yes,41
3611,DESCRIPTION OF GOODS QUANTITY MT UNIT PRICE US...,PC RESIN,4A,"[PC RESIN, RESIN , INA ]",yes,4A
3612,DESCRIPTION OF GOODS QUANTITY MT UNIT PRICE US...,PC RESIN,4A,"[PC RESIN, RESIN , INA ]",yes,4A


In [106]:
result['EXPNO'] = [ str(i).strip() for i in result['EXPNO']]
result['預測部門代號'] = [ str(i).strip() for i in result['預測部門代號']]

In [107]:
result[result['EXPNO']==result['預測部門代號']]

,string_X_train,Y_label,EXPNO,predict,correct,預測部門代號
0,MASS PVC RESIN B-57 QUANTITY 175 MT AT 1300 US...,MASS PVC RESIN B-57,11,"[PVC RESIN, RESIN , PVC RESIN B-57]",yes,11
1,PHTHALIC ANHYDRIDE PA QUANTITY 306 MT UNIT PRI...,PHTHALIC ANHYDRIDE,27,"[PHTHALIC ANHYDRIDE , PA , PHTHALIC ANHYDRIDE...",yes,27
2,COMMODITY LLDPE TAISOX 3470 QUANTITY 320 MT 2 ...,LLDPE TAISOX,18,"[LLDPE TAISOX, TAISOX ]",yes,18
3,COMMODITY AMOUNT POLYESTER TEXTURED YARN HS CO...,POLYESTER PARTIALY ORIENTED YARN,25,"[POLYESTER TEXTURED YARN, POLYESTER PARTIALY O...",yes,25
4,COMMODITY AMOUNT POLYESTER TEXTURED YARN HS CO...,POLYESTER FILAMENT YARN,25,"[POLYESTER TEXTURED YARN, POLYESTER PARTIALY O...",yes,25
...,...,...,...,...,...,...
3609,COMMODITY MONO ETHYLENE GLYCOL x 000 D QUANTIT...,MONO ETHYLENE GLYCOL,2A,"[MONO ETHYLENE GLYCOL, ETHYLENE , INA ]",yes,2A
3610,CFR KOBE JAPAN x 000 D VISCOSE RAYON STAPLE FI...,VISCOSE RAYON STAPLE FIBER,41,"[ RAYON , VISCOSE RAYON STAPLE FIBER, RAYON ST...",yes,41
3611,DESCRIPTION OF GOODS QUANTITY MT UNIT PRICE US...,PC RESIN,4A,"[PC RESIN, RESIN , INA ]",yes,4A
3612,DESCRIPTION OF GOODS QUANTITY MT UNIT PRICE US...,PC RESIN,4A,"[PC RESIN, RESIN , INA ]",yes,4A


In [108]:
result[result['EXPNO']!=result['預測部門代號']]

,string_X_train,Y_label,EXPNO,predict,correct,預測部門代號
9,300000 BB LS - 10 P C T OF GASOIL 10 PPM UNIT ...,GASOIL,61,[ GASOIL ],yes,60
10,300000 BB LS - 10 P C T OF GASOIL 10 PPM UNIT ...,GASOIL,61,[ GASOIL ],yes,60
13,300000 BB LS - 10 P C T OF GASOIL 10 PPM UNIT ...,GASOIL,61,[ GASOIL ],yes,60
14,PRODUCT TETRAHYDROFURAN 998 PCT MIN QUANTITY 1...,TETRAHYDROFURAN,2P,[ TETRAHYDROFURAN ],yes,24
26,COMMODITY MONOETHYLENE GLYCOL MEG FIBER GRADE ...,MONOETHYLENE GLYCOL MEG FIBER GRADE,2A,[INA ],no,27
...,...,...,...,...,...,...
3541,QUANTITY 36 MTS OF POM FORMOCON AS FOLLOWS FM0...,PE,1P,[PROFORMA],no,11
3542,QUANTITY 36 MTS OF POM FORMOCON AS FOLLOWS FM0...,PE,1P,[PROFORMA],no,11
3549,COMMODITY MONOETHYLENE GLYCOL FIBER GRADE H SN...,MONOETHYLENE GLYCOL FIBER GRADE,2A,[MONOETHYLENE],no,11
3589,COMMODITY MONOETHYLENE GLYCOL x 000 D QUANTITY...,ETHYLENE,2A,[INA ],no,27


In [113]:
品名2代號['GASOIL'] # EXPNO是61 寶典是60

60

In [114]:
len(result[result['EXPNO']==result['預測部門代號']])/len(result)

0.7794164796409105

In [115]:
result.to_csv('submit_0726.csv')